In [669]:
## Only run if you need the news headlines! This will download the headlines from the internet into a new csv every time!!!

## !wget https://bitbucket.org/newsstuff/headlinesdata/raw/3d468c6c414ffb591f6e685644da25aab1621bdf/stock_news.csv

--2024-05-04 18:16:47--  https://bitbucket.org/newsstuff/headlinesdata/raw/3d468c6c414ffb591f6e685644da25aab1621bdf/stock_news.csv
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2227283 (2.1M) [text/plain]
Saving to: ‘stock_news.csv.41’

stock_news.csv.41   100%[===================>]   2.12M  12.5MB/s    in 0.2s    

2024-05-04 18:16:48 (12.5 MB/s) - ‘stock_news.csv.41’ saved [2227283/2227283]



In [670]:
import pandas as pd

df = pd.read_csv('stock_news.csv')


   Unnamed: 0                                           headline     label
0           0  Markets Close Mostly Lower Again; ROST, PANW R...  Negative
1           1  Gap plummets on earnings miss, cuts full-year ...  Negative
2           2  Billionaire Ken Fisher is Selling These 10 Stocks  Negative
3           3          Corning net income drops 13%, shares fall  Negative
4           4  Internet Explorer shutdown to cause Japan prob...  Negative


In [671]:
del df['Unnamed: 0']

In [672]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.sentiment import vader

In [673]:
v_t = TfidfVectorizer()
bad_vc = CountVectorizer()
good_vc = CountVectorizer()
neutral_vc = CountVectorizer()
sia = vader.SentimentIntensityAnalyzer()

In [674]:
df['headline'] = df['headline'].apply(lambda x: x.lower())  

df_good = df[df['label'] == 'Positive']
df_bad = df[df['label'] == 'Negative']
df_neutral = df[df['label'] == 'Neutral']

In [675]:
v_t.fit(df['headline'])

good_vc.fit(df_good['headline'])
bad_vc.fit(df_bad['headline'])
neutral_vc.fit(df_neutral['headline'])

CountVectorizer()

In [676]:
df_good_sia = df_good['headline'].apply(lambda x: sia.polarity_scores(x))
df_bad_sia = df_bad['headline'].apply(lambda x: sia.polarity_scores(x))
df_neutral_sia = df_neutral['headline'].apply(lambda x: sia.polarity_scores(x))

In [677]:
df_good_bad_vc = bad_vc.transform(df_good['headline'])
df_bad_bad_vc = bad_vc.transform(df_bad['headline'])
df_neutral_bad_vc = bad_vc.transform(df_neutral['headline'])

df_good_good_vc = good_vc.transform(df_good['headline'])
df_bad_good_vc = good_vc.transform(df_bad['headline'])
df_neutral_good_vc = good_vc.transform(df_neutral['headline'])

df_good_neutral_vc = neutral_vc.transform(df_good['headline'])
df_bad_neutral_vc = neutral_vc.transform(df_bad['headline'])
df_neutral_neutral_vc = neutral_vc.transform(df_neutral['headline'])

df_good_v_t = v_t.transform(df_good['headline'])
df_bad_v_t = v_t.transform(df_bad['headline'])
df_neutral_v_t = v_t.transform(df_neutral['headline'])

In [678]:
good_features = np.hstack([df_good_sia.apply(pd.Series).values, df_good_bad_vc.toarray(), df_good_good_vc.toarray(), df_good_neutral_vc.toarray(), df_good_v_t.toarray()])
bad_features = np.hstack([df_bad_sia.apply(pd.Series).values, df_bad_bad_vc.toarray(), df_bad_good_vc.toarray(), df_bad_neutral_vc.toarray(), df_bad_v_t.toarray()])
neutral_features = np.hstack([df_neutral_sia.apply(pd.Series).values, df_neutral_bad_vc.toarray(), df_neutral_good_vc.toarray(), df_neutral_neutral_vc.toarray(), df_neutral_v_t.toarray()])

In [679]:
good_labels = np.ones(good_features.shape[0])
bad_labels = np.full(bad_features.shape[0], -1)
neutral_labels = np.zeros(neutral_features.shape[0])

In [680]:
from sklearn.model_selection import train_test_split

X = np.vstack([good_features, bad_features, neutral_features])

y = np.hstack([good_labels, bad_labels, neutral_labels])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [681]:
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss = 'log_loss', penalty = 'l2', alpha = 1.75e-3, learning_rate='optimal', random_state = 42, max_iter = 4, tol = None, early_stopping=True, validation_fraction=0.1, n_iter_no_change=15,class_weight='balanced')

X_initial_fit = X_train[:int(X_train.shape[0] * 0.1)]
y_initial_fit = y_train[:int(y_train.shape[0] * 0.1)]

clf.fit(X_initial_fit, y_initial_fit)


SGDClassifier(alpha=0.00175, class_weight='balanced', early_stopping=True,
              loss='log_loss', max_iter=4, n_iter_no_change=15, random_state=42,
              tol=None)

In [682]:
from sklearn.metrics import accuracy_score

y_pred = clf.predict(X_test)

print(accuracy_score(y_test, y_pred))

0.6528846153846154


In [683]:
def partial_fit_step_train():
    for i in range(10):
        X_partial_fit = X_train[int(X_train.shape[0] * 0.05) * i:int(X_train.shape[0] * 0.1) * (i + 1)]
        y_partial_fit = y_train[int(y_train.shape[0] * 0.05) * i:int(y_train.shape[0] * 0.1) * (i + 1)]
    
        clf.partial_fit(X_partial_fit, y_partial_fit)
        y_pred = clf.predict(X_test)
    
        print(accuracy_score(y_test, y_pred))

In [684]:
partial_fit_step_train()

0.6542307692307693
0.6821153846153846
0.6676923076923077
0.693076923076923
0.6990384615384615
0.7009615384615384
0.6967307692307693
0.7042307692307692
0.7071153846153846
0.7121153846153846


In [685]:
def test():   
    y_pred = clf.predict(X_test)
    
    print(accuracy_score(y_test, y_pred))

In [686]:
test()

0.7121153846153846


In [718]:
with open ('headLines.csv') as headlines:
    headlines = headlines.read().split('\n')
    
headlines

['Activist Groups Trained Students for Months Before Campus Protests',
 'Jobs Data Boost Hopes of a Late-Summer Rate Cut',
 '"While U.S. Debates Free Speech, Pro-Hamas Rhetoric in France Is a Crime"',
 'U.S. Shuffles Military Assets in Mideast After Gulf Pushback',
 'Texas Democrat Henry Cuellar Indicted on Corruption Charges',
 'Hope Hicks Recalls Campaign Crisis After Emergence of ‘Access Hollywood’ Tape',
 'There’s More to Buffett’s Game Than Just Picking Great Stocks',
 '"The CEO Who Hired His Wife, Gave His Dog a Title, and Brought Down a Bank"',
 'Elon Musk Tries to Make Himself Indispensable at Tesla Ahead of Pay Vote',
 'America’s War Machine Runs on Rare-Earth Magnets. China Owns That Market.',
 'Why You May Soon Be Drinking Synthetic Coffee',
 'Backdoor Roth IRAs Are Promising—and Perilous',
 'Skip the Summer Crowds in Europe. Try These Underrated Destinations Instead.',
 'How I Got My Attention Span Back',
 'Colleges Have a New Source of Protest on Their Hands: Irate Parents

In [719]:
headlines_good_vc = good_vc.transform(headlines)
headlines_bad_vc = bad_vc.transform(headlines)
headlines_neutral_vc = neutral_vc.transform(headlines)

headlines_v_t = v_t.transform(headlines)

headlines_sia = pd.Series([sia.polarity_scores(x) for x in headlines])

headlines_features = np.hstack([headlines_sia.apply(pd.Series).values, headlines_bad_vc.toarray(), headlines_good_vc.toarray(), headlines_neutral_vc.toarray(), headlines_v_t.toarray()])

headlines_pred = clf.predict(headlines_features)

headlines_pred

headlines = pd.DataFrame({'headline': headlines, 'label': headlines_pred})

In [720]:
headlines

,headline,label
0,Activist Groups Trained Students for Months Before Campus Protests,0.0
1,Jobs Data Boost Hopes of a Late-Summer Rate Cut,1.0
2,"""While U.S. Debates Free Speech, Pro-Hamas Rhetoric in France Is a Crime""",-1.0
3,U.S. Shuffles Military Assets in Mideast After Gulf Pushback,1.0
4,Texas Democrat Henry Cuellar Indicted on Corruption Charges,-1.0
...,...,...
162,Analyst Report: Public Service Enterprise Group Incorporated,-1.0
163,Starbucks Is Running Out of Americans to Drink Its Expensive Coffee,-1.0
164,Trump Media auditor shut down and charged with ‘massive fraud’,-1.0
165,,-1.0


In [721]:
headlines['label'] = headlines['label'].map({1: 'Positive', -1: 'Negative', 0: 'Neutral'})

In [722]:
prob_headlines = clf.predict_proba(headlines_features)

In [723]:
# Extract the maximum probability from each row in prob_headlines
confidence = np.max(prob_headlines, axis=1)

# Create a new DataFrame
df_prob_headlines = pd.DataFrame({
    'headline': headlines['headline'],
    'prediction': headlines['label'],
    'confidence': confidence
})

# Assign 'Neutral' to 'prediction' where 'confidence' is less than 0.5
df_prob_headlines.loc[df_prob_headlines['confidence'] < 0.5, 'prediction'] = 'Neutral'

In [724]:
df_prob_headlines

,headline,prediction,confidence
0,Activist Groups Trained Students for Months Before Campus Protests,Neutral,0.578836
1,Jobs Data Boost Hopes of a Late-Summer Rate Cut,Positive,0.528090
2,"""While U.S. Debates Free Speech, Pro-Hamas Rhetoric in France Is a Crime""",Neutral,0.467593
3,U.S. Shuffles Military Assets in Mideast After Gulf Pushback,Neutral,0.433167
4,Texas Democrat Henry Cuellar Indicted on Corruption Charges,Neutral,0.458765
...,...,...,...
162,Analyst Report: Public Service Enterprise Group Incorporated,Neutral,0.452902
163,Starbucks Is Running Out of Americans to Drink Its Expensive Coffee,Neutral,0.475452
164,Trump Media auditor shut down and charged with ‘massive fraud’,Negative,0.782213
165,,Neutral,0.460335


In [725]:
df_prob_headlines = df_prob_headlines[df_prob_headlines['prediction'] != 'Neutral']

In [726]:
df_prob_headlines

,headline,prediction,confidence
1,Jobs Data Boost Hopes of a Late-Summer Rate Cut,Positive,0.528090
5,Hope Hicks Recalls Campaign Crisis After Emergence of ‘Access Hollywood’ Tape,Negative,0.614395
7,"""The CEO Who Hired His Wife, Gave His Dog a Title, and Brought Down a Bank""",Negative,0.714010
8,Elon Musk Tries to Make Himself Indispensable at Tesla Ahead of Pay Vote,Positive,0.582099
9,America’s War Machine Runs on Rare-Earth Magnets. China Owns That Market.,Negative,0.628321
...,...,...,...
157,DJT Stock Falls. SEC Charges Trump Media Auditing Firm With ‘Massive Fraud.’,Negative,0.715673
158,"""The US could give homeowners a $980 billion stimulus at no additional cost, 'Oracle of Wall Street' says""",Positive,0.511566
159,Suze Orman Calls The 4% Retirement Rule 'Very Dangerous' In Today's Challenging Economic Climate,Negative,0.671411
160,A 64-year-old 'peak boomer' worries his savings and Social Security won't be enough to retire: 'My biggest fear is finding myself at 75 standing at the door at Walmart.',Positive,0.512605


In [727]:

df_prob_headlines.loc[:, 'prediction'] = df_prob_headlines['prediction'].map({'Positive': 1.0, 'Negative': -1.0})

sum = 0

for _, row in df_prob_headlines.iterrows():
    sum += row['confidence'] * row['prediction']


['Positive' 'Negative']
[1.0 -1.0]
Confidence: 0.5280896894917915, Prediction: 1.0
Confidence: 0.6143954800364625, Prediction: -1.0
Confidence: 0.7140096917222317, Prediction: -1.0
Confidence: 0.5820992167200028, Prediction: 1.0
Confidence: 0.6283208098419564, Prediction: -1.0
Confidence: 0.6696707410596531, Prediction: -1.0
Confidence: 0.5697365170675127, Prediction: -1.0
Confidence: 0.7019238288759654, Prediction: 1.0
Confidence: 0.5801254188118594, Prediction: -1.0
Confidence: 0.5611007760814087, Prediction: -1.0
Confidence: 0.6663863880171712, Prediction: 1.0
Confidence: 0.5744500578906401, Prediction: 1.0
Confidence: 0.893969294781065, Prediction: -1.0
Confidence: 0.6248212532094748, Prediction: -1.0
Confidence: 0.5190803354900003, Prediction: -1.0
Confidence: 0.5366222563974081, Prediction: 1.0
Confidence: 0.5512863828937035, Prediction: 1.0
Confidence: 0.7079378252684797, Prediction: 1.0
Confidence: 0.5583837878508914, Prediction: 1.0
Confidence: 0.6258081769438713, Prediction: 

In [728]:
sum

-1.7679495495820925

In [734]:
import pickle as p
try:
    sia = p.load(open('vaderNews.pkl', 'rb'))
    
except Exception as e:
    print("g")
    sia = vader.SentimentIntensityAnalyzer()
    p.dump(sia, open('vaderNews.pkl', 'wb'))


In [736]:
try: 
    good_vc = p.load(open('good_vc.pkl', 'rb'))
    bad_vc = p.load(open('bad_vc.pkl', 'rb'))
    neutral_vc = p.load(open('neutral_vc.pkl', 'rb'))
    v_t = p.load(open('v_t.pkl', 'rb'))
    
except Exception as e:
    print("g")
    good_vc = CountVectorizer()
    bad_vc = CountVectorizer()
    neutral_vc = CountVectorizer()
    v_t = TfidfVectorizer()
    p.dump(good_vc, open('good_vc.pkl', 'wb'))
    p.dump(bad_vc, open('bad_vc.pkl', 'wb'))
    p.dump(neutral_vc, open('neutral_vc.pkl', 'wb'))
    p.dump(v_t, open('v_t.pkl', 'wb'))
    